In [ ]:
import pandas as pd
from itertools import combinations
from collections import Counter
import holoviews as hv
from holoviews import opts
import matplotlib.pyplot as plt

hv.extension('bokeh')

# Load the dataset
tmdb_data = pd.read_csv('TMDB_movie_dataset_v11.csv')

# Split the genres and handle any non-string values
tmdb_data['genres'] = tmdb_data['genres'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Filter out movies without valid genres
valid_movies = tmdb_data[tmdb_data['genres'].apply(len) > 0]

# Calculate genre co-occurrences
genre_pairs = valid_movies['genres'].apply(lambda genres: list(combinations(genres, 2)))
genre_pairs = [pair for sublist in genre_pairs for pair in sublist]
genre_pair_counts = Counter(genre_pairs)

# Prepare data for chord plot
data = []
for (genre1, genre2), count in genre_pair_counts.items():
    data.append([genre1, genre2, count])

# Create a DataFrame from the data
chord_data = pd.DataFrame(data, columns=['source', 'target', 'value'])

# Custom colormap excluding green cuz I don't like it on my plot
custom_colormap = [
    "#1f77b4",  # Blue
    "#ff7f0e",  # Orange
    "#d62728",  # Red
    "#9467bd",  # Purple
    "#8c564b",  # Brown
    "#e377c2",  # Pink
    "#7f7f7f",  # Gray
    "#17becf",  # Cyan
    "#aec7e8",  # Light Blue
    "#ffbb78",  # Light Orange
    "#ff9896",  # Light Red
    "#c5b0d5",  # Light Purple
    "#c49c94",  # Light Brown
    "#f7b6d2",  # Light Pink
    "#c7c7c7",  # Light Gray
    "#9edae5"   # Light Cyan
]

# Create nodes
unique_genres = pd.DataFrame({'index': list(set(chord_data['source']).union(set(chord_data['target'])))})
nodes = hv.Dataset(unique_genres)

# Convert the DataFrame to a Chord element
chord = hv.Chord((chord_data, nodes))

# Customize the plot for better visibility
chord.opts(
    opts.Chord(cmap=custom_colormap, edge_color=hv.dim('source').str(), labels='index', node_color=hv.dim('index').str(),
               edge_cmap=custom_colormap, edge_alpha=0.8, node_size=20, height=800, width=800,
               label_text_font_size='8pt', node_line_color='black'))

# Display the plot
hv.save(chord, 'genre_chord_plot.html', fmt='html')
